In [2]:
import hashlib
import copy

In [3]:
class Node:
    def __init__(self, valor):
        
        self.valor = valor
        self.string = valor
        self.lado = None
        self.brother = None
        self.parent = None
        self.izquierda = None
        self.derecha = None

In [39]:
class MerkleTree:
    def __init__(self, strings, hash_func):
        self.hash_func = hash_func
        self.leaves = []
        self.root = self.construir(strings)
        

    def get_root(self):
        return self.root

    def get_proof_for(self, item):

        for leave in self.leaves:
            pruebas = [(leave.valor,leave.lado)]
            pruebas_texto = [(leave.valor,leave.lado)]

            if self.hash_func(item.encode('utf-8')).hexdigest() == leave.hash:
                nodo_actual = leave
                while nodo_actual.parent:
                    pruebas.append((nodo_actual.brother.hash,nodo_actual.brother.lado))
                    pruebas_texto.append((nodo_actual.brother.string,nodo_actual.brother.lado))
                    nodo_actual = nodo_actual.parent
                print(pruebas_texto)
                return pruebas
        return None

    def construir(self,strings):
        print_table = []
        nodos = []
        for s in strings:
            nuevo_nodo = Node(s)
            nuevo_nodo.hash = self.hash_func(s.encode('utf-8')).hexdigest()
            nodos.append(nuevo_nodo)
            print_table.append(nuevo_nodo.hash)
            self.leaves.append(nuevo_nodo)
        print(print_table)

        while len(nodos) > 1:
            cambio = []
            print_table = []
            for index in range(0, len(nodos), 2):
                izquierda = nodos[index]
                if index+1 < len(nodos):
                    derecha = nodos[index+1]
                else:
                    derecha = copy.copy(izquierda)
                derecha.lado = 'd'
                derecha.brother = izquierda
                izquierda.brother = derecha
                izquierda.lado = 'i'
                nuevo_hash = izquierda.hash + derecha.hash
                nuevo_nodo = Node(nuevo_hash)
                nuevo_nodo.string = izquierda.string + derecha.string
                nuevo_nodo.hash = self.hash_func(nuevo_hash.encode('utf-8')).hexdigest()
                nuevo_nodo.izquierda = izquierda
                nuevo_nodo.derecha = derecha
                nuevo_nodo.izquierda.parent = nuevo_nodo
                nuevo_nodo.derecha.parent = nuevo_nodo
                cambio.append(nuevo_nodo)
                print_table.append(nuevo_nodo.hash)
            print(print_table)
            nodos = cambio
        return nodos[0]


In [40]:
tree = MerkleTree(['1','2','3','4','5','6'],hashlib.sha256)


['6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d49c01e52ddb7875b4b', 'd4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f90da3a666eec13ab35', '4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce', '4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328cb08b5531fcacdabf8a', 'ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d', 'e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683']
['33b675636da5dcc86ec847b38c08fa49ff1cace9749931e0a5d4dfdbdedd808a', '13656c83d841ea7de6ebf3a89e0038fea9526bd7f686f06f7a692343a8a32dca', '43587f59c00a8e528bc7636fabaffcf70cc25afc5b4d53df797faf0dc72f6dd0']
['85df8945419d2b5038f7ac83ec1ec6b8267c40fdb3b1e56ff62f6676eb855e70', 'e83e8d06e60bcfea6f4307c12ad0600566fd91b927ed7fed2ed1e1caf1a3536f']
['058bd72c469db066d7b28c9e63e1b7b05c48df9ca23dd521afd0b6154ea47be6']


In [41]:
print(tree.get_proof_for('5'))

[('5', 'i'), ('6', 'd'), ('56', 'd'), ('1234', 'i')]
[('5', 'i'), ('e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683', 'd'), ('43587f59c00a8e528bc7636fabaffcf70cc25afc5b4d53df797faf0dc72f6dd0', 'd'), ('85df8945419d2b5038f7ac83ec1ec6b8267c40fdb3b1e56ff62f6676eb855e70', 'i')]


In [51]:
def verify(root , item , proof):
    
    if proof[0][0] == item:
        if proof[0][1] == 'i':
            actual_texto = proof[0][0] + proof[1][0]
            actual = hashlib.sha256(proof[0][0].encode('utf-8')).hexdigest()  + proof[1][0]
        else:
            actual_texto = proof[1][0] + proof[0][0]
            actual = hashlib.sha256(proof[1][0].encode('utf-8')).hexdigest()  + proof[0][0]
          
        actual = hashlib.sha256(actual.encode('utf-8')).hexdigest()

        

        for node in proof[2::]:
            if node[1] == 'i':
                actual_texto = node[0] + actual_texto
                actual = node[0] + actual
            else:
                actual_texto = actual_texto + node[0]
                actual = actual + node[0]
            
            actual = hashlib.sha256(actual.encode('utf-8')).hexdigest()
        if actual == root:
            return True
        return False
            



In [52]:
print(tree.root.hash)


058bd72c469db066d7b28c9e63e1b7b05c48df9ca23dd521afd0b6154ea47be6


In [55]:
print(verify('058bd72c469db066d7b28c9e63e1b7b05c48df9ca23dd521afd0b6154ea47be6' ,  '5',[('5', 'i'), ('e7f6c011776e8db7cd330b54174fd76f7d0216b612387a5ffcfb81e6f0919683', 'd'), ('43587f59c00a8e528bc7636fabaffcf70cc25afc5b4d53df797faf0dc72f6dd0', 'd'), ('85df8945419d2b5038f7ac83ec1ec6b8267c40fdb3b1e56ff62f6676eb855e70', 'i')]))

True
